In [3]:
import requests
import json
from sentence_transformers import SentenceTransformer

from os import listdir
from os.path import isfile, join

import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

#from lyrics_api import *
import csv

In [4]:
'''
with open('test.csv', 'w', encoding='UTF8', newline='') as f:
    writer = csv.writer(f)

    for i in range(1000):
    
        val = input("Y/N")
        
        if val == 'N':
            break
    
        header = ['fuck', 'my', 'ass']
        writer.writerow(header)
'''


'\nwith open(\'test.csv\', \'w\', encoding=\'UTF8\', newline=\'\') as f:\n    writer = csv.writer(f)\n\n    for i in range(1000):\n    \n        val = input("Y/N")\n        \n        if val == \'N\':\n            break\n    \n        header = [\'fuck\', \'my\', \'ass\']\n        writer.writerow(header)\n'

In [11]:
def getLyrics(artistName, trackName):

    #CREATE THE URL
    baseUrl = "https://api.musixmatch.com/ws/1.1/"

    apiKey = "&apikey=bd88be6fb629c354c5ca281296ac8c47"

    lyricsMatcher = "matcher.lyrics.get"

    formatUrl = "?format=json&callback=callback"

    artistSearch = "&q_artist="

    trackSearch = "&q_track="

    #MAKE THE API CALL
    apiCall = baseUrl + lyricsMatcher + formatUrl + artistSearch + artistName + trackSearch + trackName + apiKey

    request = requests.get(apiCall)

    data = request.json()

    data = data['message']['body']
    
    if data == []:
        return []

    lyrics = data['lyrics']['lyrics_body']
    
    #at this point lyrics is a string and I turn it into a list.
    
    #PROCESS THE DATA
    lyrics = lyrics.split('\n')
    
    lyrics = list(filter(lambda a: a != '', lyrics))
    
    lyrics = lyrics[:-2]
    
    #output = ""
    
    '''
    for sentence in lyrics:
        output = output + sentence + ". "
        
    output = output[:-1]
    '''
    
    return lyrics

In [12]:
#getLyrics('n', 'Wasd')

[]

In [11]:
cid="715ae75406e24c24a3641929260ca343"
secret="e60a686258c646b7a4d3f9636b3235db"

client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret)
sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager)

In [12]:
def getEmbedding(lyrics):
    model = SentenceTransformer('all-MiniLM-L6-v2')

    embeddings = model.encode(lyrics)

    return embeddings

In [13]:
#you can download the dataset here:
#https://www.aicrowd.com/challenges/spotify-million-playlist-dataset-challenge/dataset_files
#you do need an account however.

mypath = '../../musicdata'

onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]

BIGDATA = {}

with open('songs.csv', 'w', encoding='UTF8', newline='') as f:
    writer = csv.writer(f)
    
    header = ['Track Name', 'Artist Name', 'Album Name', 'Artist URI', 'Album URI', 'Genre', 'Tempo', 'Danceability', 'Loudness', 'Lyrics', 'Track Uri']
    writer.writerow(header)

    for fileName in onlyfiles:
        f = open(mypath + '/' + fileName)

        data = json.load(f)

        playlists = data['playlists']

        for playlist in playlists:
            tracks = playlist['tracks']
            
            if len(BIGDATA) >= 1000000:
                break

            for track in tracks:
                uri = track['track_uri']

                if uri in BIGDATA:
                    continue

                BIGDATA[uri] = {}

                BIGDATA[uri]['trackName'] = track['track_name']
                BIGDATA[uri]['artistName'] = track['artist_name']
                BIGDATA[uri]['albumName'] = track['album_name']

                BIGDATA[uri]['artistUri'] = track['artist_uri']
                BIGDATA[uri]['albumUri'] = track['album_uri']

                BIGDATA[uri]['genres'] = sp.artist(BIGDATA[uri]['artistUri'])["genres"]
                BIGDATA[uri]['tempo'] = sp.audio_features(uri)[0]['tempo']
                BIGDATA[uri]['danceability'] = sp.audio_features(uri)[0]['danceability']
                BIGDATA[uri]['loudness'] = sp.audio_features(uri)[0]['loudness']

                BIGDATA[uri]['lyrics'] = getLyrics(BIGDATA[uri]['artistName'], BIGDATA[uri]['trackName'])
                
                row = list(BIGDATA[uri].values())
                row.append(uri)
                
                writer.writerow(row)

        f.close()

KeyboardInterrupt: 

In [26]:
uri = data['playlists'][3]['tracks'][0]['track_uri']

sp.audio_features(uri)[0]

{'danceability': 0.438,
 'energy': 0.178,
 'key': 7,
 'loudness': -20.392,
 'mode': 0,
 'speechiness': 0.0445,
 'acousticness': 0.908,
 'instrumentalness': 0.752,
 'liveness': 0.0688,
 'valence': 0.166,
 'tempo': 114.515,
 'type': 'audio_features',
 'id': '4WJ7UMD4i6DOPzyXU5pZSz',
 'uri': 'spotify:track:4WJ7UMD4i6DOPzyXU5pZSz',
 'track_href': 'https://api.spotify.com/v1/tracks/4WJ7UMD4i6DOPzyXU5pZSz',
 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/4WJ7UMD4i6DOPzyXU5pZSz',
 'duration_ms': 428560,
 'time_signature': 4}